In [170]:
import pandas as pd

loc_data = pd.read_csv('source/cherryblossom_loc_selected_station.csv',
                     sep=',', engine='python')

## 시간별 지하철 역 입구 통행자 정보 가져오기

In [173]:
import requests
import os
from dotenv import load_dotenv
load_dotenv()

# 수정
api_key = os.getenv("SKT_API_KEY")

## 역 코드로 출구 통행량 가져오기

In [174]:
def get_user_cnt_by_station_code(station_code, date):
    url = f"https://apis.openapi.sk.com/puzzle/subway/exit/raw/hourly/stations/{station_code}"

    headers = {
        "accept": "application/json",
        "Content-Type": "application/json",
        "appkey": api_key
    }

    params = {
        "gender": "all",
        "ageGrp": "all",
        "date": date
    }

    response = requests.get(url, headers=headers, params=params)
    
    return response

## 낮(09-16), 밤(17-23) 각각의 총 출구 통행량 구하기

In [192]:
def caculate_passenger_data(data, user_day_cnt_list, user_night_cnt_list):
    user_cnt_day = 0
    user_cnt_night = 0
    
    # 통행자 수 계산
    for raw in data['contents']['raw']:
        if raw['exit'] == str(row['exit_num']):
            if raw['datetime'] >= (str(date)+"090000") and raw['datetime'] < (str(date)+"170000"):
                user_cnt_day += (int(raw['userCount']) if raw['userCount'] != None else 0)
            elif raw['datetime'] >= (str(date)+"170000"):
                user_cnt_night += (int(raw['userCount']) if raw['userCount'] != None else 0)

    # 리스트에 저장
    user_cnt_day_list.append(user_cnt_day)
    user_cnt_night_list.append(user_cnt_night)

In [197]:
date = 20230401

for i in range(1, 8):
    user_cnt_day_list = []
    user_cnt_night_list = []
    
    for index, row in loc_data.iterrows():
        data = get_user_cnt_by_station_code(row['station_code'], date).json()
        caculate_passenger_data(data, user_day_cnt_list, user_night_cnt_list)
    # 통행량 데이터
    loc_data_temp = loc_data
    loc_data_temp['user_cnt_day'] = user_cnt_day_list
    loc_data_temp['user_cnt_night'] = user_cnt_night_list
    
    file_path = 'source/cherryblossom_final_{}.csv'.format(date)
    loc_data_temp.to_csv(file_path,
                      encoding='cp949', mode='w', index=False)
    date += 1

국회의사당역
day 20230601090000 402 2
day 20230601100000 121 2
day 20230601110000 162 2
day 20230601120000 148 2
day 20230601130000 150 2
day 20230601140000 147 2
day 20230601150000 242 2
day 20230601160000 223 2
night 20230601170000 410 2
night 20230601180000 705 2
night 20230601190000 248 2
night 20230601200000 157 2
night 20230601210000 163 2
night 20230601220000 118 2
night 20230601230000 0 2
천호역
day 20230601090000 315 3
day 20230601100000 110 3
day 20230601110000 139 3
day 20230601120000 122 3
day 20230601130000 111 3
day 20230601140000 116 3
day 20230601150000 148 3
day 20230601160000 138 3
night 20230601170000 309 3
night 20230601180000 322 3
night 20230601190000 249 3
night 20230601200000 185 3
night 20230601210000 127 3
night 20230601220000 93 3
night 20230601230000 13 3
금천구청역
day 20230601090000 549 1
day 20230601100000 393 1
day 20230601110000 352 1
day 20230601120000 380 1
day 20230601130000 433 1
day 20230601140000 269 1
day 20230601150000 395 1
day 20230601160000 540 1
night 202

day 20230601090000 63 1
day 20230601100000 27 1
day 20230601110000 29 1
day 20230601120000 22 1
day 20230601130000 24 1
day 20230601140000 32 1
day 20230601150000 46 1
day 20230601160000 37 1
night 20230601170000 50 1
night 20230601180000 108 1
night 20230601190000 46 1
night 20230601200000 39 1
night 20230601210000 27 1
night 20230601220000 31 1
night 20230601230000 None 1
외대앞역
day 20230601090000 29 3
day 20230601100000 8 3
day 20230601110000 16 3
day 20230601120000 16 3
day 20230601130000 26 3
day 20230601140000 19 3
day 20230601150000 16 3
day 20230601160000 14 3
night 20230601170000 26 3
night 20230601180000 47 3
night 20230601190000 36 3
night 20230601200000 36 3
night 20230601210000 14 3
night 20230601220000 20 3
night 20230601230000 0 3
국회의사당역
day 20230602090000 363 2
day 20230602100000 133 2
day 20230602110000 110 2
day 20230602120000 100 2
day 20230602130000 162 2
day 20230602140000 221 2
day 20230602150000 161 2
day 20230602160000 204 2
night 20230602170000 399 2
night 202306

day 20230602090000 41 1
day 20230602100000 28 1
day 20230602110000 31 1
day 20230602120000 30 1
day 20230602130000 29 1
day 20230602140000 12 1
day 20230602150000 26 1
day 20230602160000 47 1
night 20230602170000 62 1
night 20230602180000 76 1
night 20230602190000 45 1
night 20230602200000 18 1
night 20230602210000 16 1
night 20230602220000 16 1
night 20230602230000 0 1
외대앞역
day 20230602090000 33 3
day 20230602100000 12 3
day 20230602110000 16 3
day 20230602120000 10 3
day 20230602130000 0 3
day 20230602140000 18 3
day 20230602150000 6 3
day 20230602160000 24 3
night 20230602170000 31 3
night 20230602180000 42 3
night 20230602190000 20 3
night 20230602200000 27 3
night 20230602210000 10 3
night 20230602220000 12 3
night 20230602230000 0 3
국회의사당역
day 20230603090000 43 2
day 20230603100000 54 2
day 20230603110000 88 2
day 20230603120000 67 2
day 20230603130000 76 2
day 20230603140000 88 2
day 20230603150000 95 2
day 20230603160000 87 2
night 20230603170000 87 2
night 20230603180000 63 2


day 20230603090000 228 1
day 20230603100000 211 1
day 20230603110000 234 1
day 20230603120000 238 1
day 20230603130000 231 1
day 20230603140000 180 1
day 20230603150000 230 1
day 20230603160000 254 1
night 20230603170000 270 1
night 20230603180000 196 1
night 20230603190000 187 1
night 20230603200000 155 1
night 20230603210000 202 1
night 20230603220000 158 1
night 20230603230000 51 1
양평역
day 20230603090000 31 1
day 20230603100000 15 1
day 20230603110000 18 1
day 20230603120000 29 1
day 20230603130000 21 1
day 20230603140000 36 1
day 20230603150000 18 1
day 20230603160000 48 1
night 20230603170000 26 1
night 20230603180000 34 1
night 20230603190000 24 1
night 20230603200000 18 1
night 20230603210000 21 1
night 20230603220000 14 1
night 20230603230000 0 1
외대앞역
day 20230603090000 24 3
day 20230603100000 15 3
day 20230603110000 16 3
day 20230603120000 20 3
day 20230603130000 12 3
day 20230603140000 27 3
day 20230603150000 6 3
day 20230603160000 20 3
night 20230603170000 18 3
night 2023060

day 20230604090000 54 1
day 20230604100000 52 1
day 20230604110000 30 1
day 20230604120000 35 1
day 20230604130000 43 1
day 20230604140000 65 1
day 20230604150000 56 1
day 20230604160000 52 1
night 20230604170000 44 1
night 20230604180000 49 1
night 20230604190000 44 1
night 20230604200000 35 1
night 20230604210000 45 1
night 20230604220000 29 1
night 20230604230000 0 1
미아사거리역
day 20230604090000 142 1
day 20230604100000 137 1
day 20230604110000 135 1
day 20230604120000 111 1
day 20230604130000 151 1
day 20230604140000 144 1
day 20230604150000 164 1
day 20230604160000 217 1
night 20230604170000 190 1
night 20230604180000 156 1
night 20230604190000 111 1
night 20230604200000 128 1
night 20230604210000 152 1
night 20230604220000 99 1
night 20230604230000 28 1
양평역
day 20230604090000 20 1
day 20230604100000 14 1
day 20230604110000 18 1
day 20230604120000 49 1
day 20230604130000 30 1
day 20230604140000 19 1
day 20230604150000 28 1
day 20230604160000 12 1
night 20230604170000 20 1
night 20230

day 20230605090000 156 1
day 20230605100000 102 1
day 20230605110000 132 1
day 20230605120000 97 1
day 20230605130000 89 1
day 20230605140000 101 1
day 20230605150000 90 1
day 20230605160000 148 1
night 20230605170000 172 1
night 20230605180000 185 1
night 20230605190000 120 1
night 20230605200000 88 1
night 20230605210000 79 1
night 20230605220000 54 1
night 20230605230000 37 1
화랑대역
day 20230605090000 106 1
day 20230605100000 50 1
day 20230605110000 47 1
day 20230605120000 47 1
day 20230605130000 45 1
day 20230605140000 30 1
day 20230605150000 46 1
day 20230605160000 58 1
night 20230605170000 54 1
night 20230605180000 173 1
night 20230605190000 100 1
night 20230605200000 60 1
night 20230605210000 55 1
night 20230605220000 54 1
night 20230605230000 22 1
미아사거리역
day 20230605090000 281 1
day 20230605100000 145 1
day 20230605110000 143 1
day 20230605120000 165 1
day 20230605130000 80 1
day 20230605140000 122 1
day 20230605150000 141 1
day 20230605160000 223 1
night 20230605170000 254 1
nig

day 20230606090000 61 1
day 20230606100000 52 1
day 20230606110000 46 1
day 20230606120000 36 1
day 20230606130000 42 1
day 20230606140000 48 1
day 20230606150000 52 1
day 20230606160000 36 1
night 20230606170000 51 1
night 20230606180000 28 1
night 20230606190000 35 1
night 20230606200000 44 1
night 20230606210000 23 1
night 20230606220000 53 1
night 20230606230000 6 1
양재시민의숲역
day 20230606090000 76 1
day 20230606100000 135 1
day 20230606110000 69 1
day 20230606120000 93 1
day 20230606130000 79 1
day 20230606140000 134 1
day 20230606150000 79 1
day 20230606160000 112 1
night 20230606170000 104 1
night 20230606180000 75 1
night 20230606190000 52 1
night 20230606200000 46 1
night 20230606210000 38 1
night 20230606220000 22 1
night 20230606230000 10 1
화랑대역
day 20230606090000 56 1
day 20230606100000 53 1
day 20230606110000 53 1
day 20230606120000 56 1
day 20230606130000 42 1
day 20230606140000 43 1
day 20230606150000 52 1
day 20230606160000 50 1
night 20230606170000 61 1
night 202306061800

day 20230607090000 498 1
day 20230607100000 296 1
day 20230607110000 244 1
day 20230607120000 274 1
day 20230607130000 277 1
day 20230607140000 322 1
day 20230607150000 369 1
day 20230607160000 416 1
night 20230607170000 576 1
night 20230607180000 717 1
night 20230607190000 338 1
night 20230607200000 260 1
night 20230607210000 259 1
night 20230607220000 159 1
night 20230607230000 None 1
방이역
day 20230607090000 92 1
day 20230607100000 56 1
day 20230607110000 33 1
day 20230607120000 38 1
day 20230607130000 58 1
day 20230607140000 56 1
day 20230607150000 45 1
day 20230607160000 61 1
night 20230607170000 107 1
night 20230607180000 123 1
night 20230607190000 100 1
night 20230607200000 46 1
night 20230607210000 59 1
night 20230607220000 64 1
night 20230607230000 16 1
양재시민의숲역
day 20230607090000 198 1
day 20230607100000 145 1
day 20230607110000 105 1
day 20230607120000 79 1
day 20230607130000 136 1
day 20230607140000 134 1
day 20230607150000 127 1
day 20230607160000 146 1
night 20230607170000 2